# Use Credit Risk Analytics

## Import Snowpark Pandas for dumping csv to Snowflake table

In [2]:
#!pip install "snowflake-connector-python[pandas]"

In [1]:
from snowflake.snowpark.session import Session
import pandas as pd
import subprocess

# Code to read Credit Risk Analytics local files and dump the same into Snowflake

In [3]:
df_install = pd.read_csv("/data/installments_payments.csv")

In [4]:
df_cc_balance = pd.read_csv("/data/credit_card_balance.csv")

In [6]:
df_prev_app = pd.read_csv("/data/previous_application.csv")

In [ ]:
df_bureau_bal = pd.read_csv("/data/bureau_balance.csv")

In [7]:
df_bureau = pd.read_csv("/data/bureau.csv")

In [1]:
df_POS_CASH = pd.read_csv("/data/POS_CASH_balance.csv")

NameError: name 'pd' is not defined

In [ ]:
print ("installments_payments ", df_install.shape)
print ("credit_card_balance ", df_cc_balance.shape)
print ("previous_application ", df_prev_app.shape)
print ("bureau_balance ", df_bureau_bal.shape)
print ("bureau ", df_bureau.shape)
print ("POS_CASH_balance ", df_POS_CASH.shape)

# Code to establish connection and  dump csv as a snowflake table in FDC setup

In [56]:
#Import all snowflake connection details from template variables.
db_user = 'MANISH'
db_password = 'Password@2023'
db_account = 'ug94937.us-east4.gcp'
db_database =  'FDC_DATA_MANISH'
db_role = 'MANISH'
db_warehouse = 'FOSFOR_SOLUTIONS_WH'
db_schema = 'PUBLIC'

In [57]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session1 = Session.builder.configs(connection_params).create()

In [58]:
session1.sql('use warehouse FOSFOR_SOLUTIONS_WH;').collect()
session1.sql('use database FDC_DATA_MANISH;').collect()
session1.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

[Row(status='Statement executed successfully.')]

In [59]:
DF_STG_CUSTOMER=session1.createDataFrame(
        STG_CUSTOMER.values.tolist(),
        schema=STG_CUSTOMER.columns.tolist())
DF_STG_CUSTOMER.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.ILF_STG_CUSTOMER")

In [61]:
DF_STG_CLAIM=session1.createDataFrame(
        STG_CLAIM.values.tolist(),
        schema=STG_CLAIM.columns.tolist())
DF_STG_CLAIM.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.ILF_STG_CLAIM")

In [62]:
DF_STG_INCIDENT=session1.createDataFrame(
        STG_INCIDENT.values.tolist(),
        schema=STG_INCIDENT.columns.tolist())
DF_STG_INCIDENT.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.ILF_STG_INCIDENT")

In [63]:
DF_STG_HARMONIZE_AI_CLAIMS=session1.createDataFrame(
        STG_HARMONIZE_AI_CLAIMS.values.tolist(),
        schema=STG_HARMONIZE_AI_CLAIMS.columns.tolist())
DF_STG_HARMONIZE_AI_CLAIMS.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.ILF_STG_HARMONIZE_AI_CLAIMS")

In [64]:
DF_STG_POLICY=session1.createDataFrame(
        STG_POLICY.values.tolist(),
        schema=STG_POLICY.columns.tolist())
DF_STG_POLICY.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.STG_POLICY")

In [65]:
DF_STG_CLAIM=session1.createDataFrame(
        STG_CLAIM.values.tolist(),
        schema=STG_CLAIM.columns.tolist())
DF_STG_CLAIM.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.STG_CLAIM")